## Lib

In [ ]:
import os
from typing import List, Literal, Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import InMemorySaver

## API key get

In [ ]:
from dotenv import load_dotenv
load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")

## LLM

In [ ]:
# get silicon api key
silicon_base_url =  "https://api.siliconflow.cn/v1"
silicon_llm_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


# init LLM mod
llm = ChatOpenAI(
    model=silicon_llm_model,
    openai_api_key=silicon_api_key,
    base_url=silicon_base_url,
    temperature=1.2,
    max_tokens=2048,
)

## TavilySearchAPIRetriever: langgraph node

In [ ]:
web_retriever = TavilySearchAPIRetriever(api_key=tavily_api_key, k=3)

question = "Describe the health benefits of taking vitamins?"

# return Document
docs = web_retriever.invoke(question)
for doc in docs:
    print("=== web search ===")
    print(f"page_content: {doc.page_content}")
    print(f"metadata: {doc.metadata}")
    print("\n")

## WikipediaRetriever: ?

In [ ]:
from langchain_community.retrievers import WikipediaRetriever

wiki_retriever = WikipediaRetriever()

question = "Describe the health benefits of taking vitamins?"
# return Document
docs = wiki_retriever.invoke(question)
for doc in docs:
    print("=== Wikipedia search ===")
    print(f"page_content: {doc.page_content}")
    print(f"metadata: {doc.metadata}")
    print("\n")

# PubMed: biomedical literature
The information content of the query is not suitable for use in general health quiz application scenarios.

In [ ]:
!pip install xmltodict

In [ ]:
from langchain_community.retrievers import PubMedRetriever

med_retriever = PubMedRetriever()

question = "Describe the health benefits of taking vitamins?"
# return Document
docs = med_retriever.invoke(question)
for doc in docs:
    print("=== PubMed search ===")
    print(f"page_content: {doc.page_content}")
    print("\n")
    print(f"metadata: {doc.metadata}")
    print("\n")

## TavilySearchResults: do not use it

In [ ]:
# Web search
web_search_tool = TavilySearchResults(
    api_key = tavily_api_key,
    k=3,
    #include_domains=["wikipedia.org"],
    #search_depth="advanced",
    include_answer=True,
    #verbose=True
)

question = "Describe the health benefits of taking vitamins?"
#docs = web_search_tool.invoke({"query": question})
docs = web_search_tool.invoke(question)
for result in docs:
    print("=== web search ===")
    print(f"url: {result['url']}")
    print(f"content: {result['content']}")
    if "answer" in result:
        print(f"answer: {result['answer']}")
    print("\n")

web_results = "\n".join([d["content"] for d in docs])
documents = Document(page_content=web_results)
print(f"{documents.page_content}")


## RetrievalQA: langchain

In [ ]:
from langchain.chains import RetrievalQA

web_retriever = TavilySearchAPIRetriever(api_key=tavily_api_key, k=3)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=web_retriever,
    chain_type="stuff"
)

question = "Describe the health benefits of taking vitamins?"
response = qa_chain.invoke(question)
#print(response)
print(response["result"])